## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Delgada,PT,37.7333,-25.6667,68.72,83,40,5.75,scattered clouds
1,1,Georgetown,MY,5.4112,100.3354,87.73,80,40,8.05,scattered clouds
2,2,Hobart,AU,-42.8794,147.3294,67.26,46,40,8.01,scattered clouds
3,3,Silale,LT,55.4906,22.1814,59.79,79,96,4.94,overcast clouds
4,4,Luderitz,NaN,-26.6481,15.1594,56.66,54,0,7.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Silale,LT,55.4906,22.1814,59.79,79,96,4.94,overcast clouds
4,4,Luderitz,NaN,-26.6481,15.1594,56.66,54,0,7.00,clear sky
7,7,Mataura,NZ,-46.1927,168.8643,50.49,97,100,8.01,overcast clouds
11,11,Nemuro,JP,43.3236,145.5750,54.03,92,100,42.79,moderate rain
12,12,Saint-Augustin,CA,51.2260,-58.6502,55.99,90,56,12.57,broken clouds
...,...,...,...,...,...,...,...,...,...,...
679,679,Rovaniemi,FI,66.5000,25.7167,58.41,100,90,5.75,mist
682,682,Camana,PE,-16.6228,-72.7111,59.04,86,69,8.84,broken clouds
688,688,Bay Roberts,CA,47.5999,-53.2648,51.33,94,0,7.05,clear sky
690,690,Strangnas,SE,59.3774,17.0312,59.41,90,35,5.97,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                107
City                   107
Country                104
Lat                    107
Lng                    107
Max Temp               107
Humidity               107
Cloudiness             107
Wind Speed             107
Current Description    107
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.count()

City_ID                104
City                   104
Country                104
Lat                    104
Lng                    104
Max Temp               104
Humidity               104
Cloudiness             104
Wind Speed             104
Current Description    104
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Silale,LT,59.79,overcast clouds,55.4906,22.1814,
7,Mataura,NZ,50.49,overcast clouds,-46.1927,168.8643,
11,Nemuro,JP,54.03,moderate rain,43.3236,145.5750,
12,Saint-Augustin,CA,55.99,broken clouds,51.2260,-58.6502,
17,Ostersund,SE,50.45,fog,63.1792,14.6357,
...,...,...,...,...,...,...,...
679,Rovaniemi,FI,58.41,mist,66.5000,25.7167,
682,Camana,PE,59.04,broken clouds,-16.6228,-72.7111,
688,Bay Roberts,CA,51.33,clear sky,47.5999,-53.2648,
690,Strangnas,SE,59.41,scattered clouds,59.3774,17.0312,


In [8]:
# 6a. Set parameters to search for hotels within 5000 meters.
parameters = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    parameters["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=parameters).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
# Replace missing hotel names with "NaN"
#hotel_df.loc[(hotel_df["Hotel Name"]==""), ["Hotel Name"]] = np.nan

# Drop rows with no hotel name
#clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.count()


City                   98
Country                98
Max Temp               98
Current Description    98
Lat                    98
Lng                    98
Hotel Name             98
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))